In [2]:
# 将数据转换成csv文件，列名为tweet和label
# twitter-2016test-A.txt中原第11065行只有一个引号，手动修正了此错误

import os
import pandas as pd
import random

sentiment2label = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

def file_to_list(fp):
    text_label = list()
    with open(fp, "r") as fin:
        for line in fin.readlines():
            splits = line.strip().split("\t")
            if file.endswith("txt"):
                text_label.append([splits[2], sentiment2label[splits[1]]])
            else:
                text_label.append([splits[3], sentiment2label[splits[2]]])
    return text_label

folder_train = "/home/featurize/data/2017_English_final/GOLD/Subtask_A"
fp_test = "/home/featurize/data/SemEval2017-task4-test/SemEval2017-task4-test.subtask-A.english.txt"

train_data = list()
for file in os.listdir(folder_train):
    if not file.endswith(("tsv", "txt")) or "README" in file:
        continue
    fp = os.path.join(folder_train, file)
    train_data += file_to_list(fp)    
test_data = file_to_list(fp_test)

df_train = pd.DataFrame(train_data, columns=["tweet", "label"])
df_test = pd.DataFrame(test_data, columns=["tweet", "label"])
df_train.to_csv("data/semeval_2017_task4_train.csv", index=False)
df_test.to_csv("data/semeval_2017_task4_test.csv", index=False)

df_train_trial = pd.DataFrame(random.sample(train_data, 1000), columns=["tweet", "label"])
df_test_trial = pd.DataFrame(random.sample(test_data, 1000), columns=["tweet", "label"])
df_train_trial.to_csv("data/semeval_2017_task4_train_trial.csv", index=False)
df_test_trial.to_csv("data/semeval_2017_task4_test_trial.csv", index=False)


In [5]:
import re

def preprocess_tweet(origin):
    tweet = re.sub(r"\\u2019", "'", origin).lower() # I\u2019m -> I'm
    tweet = re.sub(r"\\u002c", ",", tweet) # \u002c -> ,
    tweet = re.sub(r"@\S+", "@user", tweet)
    tweet = re.sub(r"https?:\/\/\S+", "http", tweet)
    tweet = re.sub(r"(www.)?[a-z]+\.(com)/?[a-z0-9]*", "http", tweet)
    return tweet

df_train_trial["tweet_preprocessed"] = df_train_trial["tweet"].apply(lambda x: preprocess_tweet(x))
df_train_trial[["tweet_preprocessed", "label"]].rename(columns={"tweet_preprocessed": "tweet"}).to_csv("data/semeval_2017_task4_train_trial_preprocessed.csv", index=False)

df_test_trial["tweet_preprocessed"] = df_test_trial["tweet"].apply(lambda x: preprocess_tweet(x))
df_test_trial[["tweet_preprocessed", "label"]].rename(columns={"tweet_preprocessed": "tweet"}).to_csv("data/semeval_2017_task4_test_trial_preprocessed.csv", index=False)

df_train["tweet_preprocessed"] = df_train["tweet"].apply(lambda x: preprocess_tweet(x))
df_train[["tweet_preprocessed", "label"]].rename(columns={"tweet_preprocessed": "tweet"}).to_csv("data/semeval_2017_task4_train_preprocessed.csv", index=False)

df_test["tweet_preprocessed"] = df_test["tweet"].apply(lambda x: preprocess_tweet(x))
df_test[["tweet_preprocessed", "label"]].rename(columns={"tweet_preprocessed": "tweet"}).to_csv("data/semeval_2017_task4_test_preprocessed.csv", index=False)

In [11]:
df_test["label"].value_counts()

1    5937
0    3972
2    2375
Name: label, dtype: int64